In [37]:
from dagster import EnvVar
from pyiceberg.catalog import Catalog, load_catalog

name = "default"
warehouse = EnvVar("NESSIE_WAREHOUSE").get_value()
branch = EnvVar("NESSIE_BRANCH").get_value()
uri = EnvVar("NESSIE_URI").get_value()
py_io_impl = "pyiceberg.io.pyarrow.PyArrowFileIO"
s3_endpoint = EnvVar("DESTINATION__FILESYSTEM__CREDENTIALS__AWS_S3_ENDPOINT").get_value()
s3_access_key_id = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_ACCESS_KEY_ID"
).get_value()
s3_secret_access_key = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_SECRET_ACCESS_KEY"
).get_value()
catalog_type = "rest"
catalog = load_catalog(
            **{
                "warehouse": warehouse,
                "uri": f"{uri}/{branch}",
                "py-io-impl": py_io_impl,
                "s3.endpoint": s3_endpoint,
                "s3.access-key-id": s3_access_key_id,
                "s3.secret-access-key": s3_secret_access_key,
                "type": catalog_type,
            },
        )

print(branch)

feat-update-schema


In [40]:
catalog.list_tables("staging")

[]

In [39]:
table = catalog.drop_table("staging.cit0day_password_files")

# Query the Data

In [29]:
table = catalog.load_table("staging.cit0day_password_files")

In [30]:
con = table.scan(limit=50).to_duckdb(table_name="cit0day_password_files")
print(pl.DataFrame(con.execute(
        "select * from cit0day_password_files"
    ).fetch_arrow_table()))

shape: (50, 6)
┌──────────────────┬──────────────────┬─────────────┬──────────────────┬────────┬──────────────────┐
│ email            ┆ username         ┆ domain      ┆ data             ┆ bucket ┆ prefix           │
│ ---              ┆ ---              ┆ ---         ┆ ---              ┆ ---    ┆ ---              │
│ str              ┆ str              ┆ str         ┆ str              ┆ str    ┆ str              │
╞══════════════════╪══════════════════╪═════════════╪══════════════════╪════════╪══════════════════╡
│ 0006diana06love0 ┆ 0006diana06love0 ┆ ya.ru       ┆ a98ce02c9b0077fa ┆ raw    ┆ extracted/Cit0da │
│ 6@ya.ru          ┆ 6                ┆             ┆ 618ae812b80698…  ┆        ┆ y Prem [_speci…  │
│ 0079817235269@ya ┆ 0079817235269    ┆ ya.ru       ┆ 91b40499a08f6d6f ┆ raw    ┆ extracted/Cit0da │
│ .ru              ┆                  ┆             ┆ 689c113bf43056…  ┆        ┆ y Prem [_speci…  │
│ 00zum.a11@ya.ru  ┆ 00zum.a11        ┆ ya.ru       ┆ 00zuma11         ┆ raw